### Active Adjustment

策略：根据现价主动调整头寸。选定一个tick长度，每次都在现价附近选定固定长度的tick。

比如我们选3为固定长度。

1. 首次提供：

    在当前价格附近选择三个tick，提供流动性。

2. 调整：

    · 当当前价格超出了选定的tick，所有的币会被转换成价格降低的那种币。取出流动性，将其中一半的token换成另一种，并在当前价格附近重新提供流动性。

    · 为了简化，我们不把收取的手续费作为头寸添加到池子中

    · 这种高频调整的策略下，取出流动性和重新添加流动性以及swap的gas fee将是很大的一块成本，我看了一下etherscan上的记录，就先都拿20美元一次来算。所以每一次调整都会花费20 * 3 usdc（如果在bnb上手续费比较低，结果可能会有所不同）

In [192]:
import sys
sys.path.append("..") 
import pandas as pd
from datetime import datetime
from importlib import reload
from poolData import swapData
from utils import utils
reload(swapData)
reload(utils)

query = swapData.SwapDataQuery()
utils = utils.utils()

In [193]:
# position info
# 选定池子： token0: usdc(decimal: 6), token1: weth(decimal: 18)
pool_id = "0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640"
query.query_positions(pool_id=pool_id, block_gte=16859000, limit=10, orderBy="owner")
decimal0 = 6
decimal1 = 18
# 初始头寸的美元价值
initial_position = 10000

# 选定时间
begin = datetime(2023, 3, 19)
end = datetime.now()

pool_id: 0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640 - type: <class 'str'>
Error: Failed to get entities from store: canceling statement due to conflict with recovery, query = /* controller='filter',application='sgd217942',route='61b1d1cdb2081a9b-260fa9e5bd2ec398',action='17071196' */
select 'Position' as entity, to_jsonb(c.*) as data from (select  c.*
  from "sgd217942"."position" c
 where c.block_range @> $1 and (lower("block_range") >= $2 and c."pool" = $3)

 order by "owner" asc, "id" asc
 limit 10) c -- binds: [17071196, 16859000, "0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640"]


In [194]:
# 获取起始时间和结束时间的价格
# 假设本金是1000刀，token0是usdc，数量是500
token0_amount = 5000
# 计算token1数量的时候需要知道价格
# 获取价格
liquidity_data = query.query_liquidity(begin=int(begin.timestamp()), end=int(end.timestamp()), pool_id="0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640")
liquidity_data
price_begin = utils.sqrtPrice2Price(int(liquidity_data["sqrtPrice"][0]))
price_end = utils.sqrtPrice2Price(int(liquidity_data["sqrtPrice"][liquidity_data.shape[0]-1]))
price_begin_real = utils.price2RealPrice(price_begin, decimal0=decimal0, decimal1=decimal1)
price_end_real = utils.price2RealPrice(price_end, decimal0=decimal0, decimal1=decimal1)
# 拿到价格之后才可以计算初始头寸
token1_amount = 5000 * price_begin_real

# 计算价格范围
tickLower = -887270
tickUpper = 887270
price_low = utils.tickIndex2Price(tickLower)
price_upper = utils.tickIndex2Price(tickUpper)
print("this is the lower price: ", price_low)
print("this is the upper price: ", price_upper)


In [71]:
# 计算liquidity 
# liquidity的计算会比较复杂，因为liquidity是会随着价格变化而实时调整的。所以我们需要算一个liquidity的时间序列出来。
# 首先我们需要一个当前价格的时间序列, 根据当前的价格计算出当前的tick，然后根据tick space来择距离当前tick最近的一个tick
liquidity_data = query.query_liquidity(begin=int(begin.timestamp()), end=int(end.timestamp()), pool_id=pool_id)
# l1 = (token0_amount * (price_low ** 0.5) * (price_upper ** 0.5)) / ((price_upper ** 0.5) - (price_low ** 0.5))
# l2 = token1_amount / ((price_upper ** 0.5) - (price_low ** 0.5))
# l = min(l1, l2)
# print("l1:", l1)
# print("l2:", l2)
# print("l:", l)

In [72]:
# 计算出当前的tick
# 把sqrtPrice转为index
liquidity_data["sqrtPrice"] = liquidity_data["sqrtPrice"].astype(float)
liquidity_data["real_price"] = liquidity_data["sqrtPrice"].astype(float).apply(lambda x: utils.sqrtPrice2Price(x))
liquidity_data["currentTickIndex"] = liquidity_data["real_price"].astype(float).apply(lambda x: utils.price2TickIndex(x))


In [97]:
# 根据tick spacing 选择距离当前价格最近的一个tick
# 不同费率的池子有不同的tick spacing
tick0 = 10 # 0.05% fee
tick1 = 60 # 0.3% fee
tick3 = 200 # 1.0% fee
# 计算出最近的tick index
liquidity_data["nearestTick"] = liquidity_data["currentTickIndex"].apply(lambda x: utils.nearestTick(x, tick0))

### 计算动态调整的liquidity

如何计算liquidity？

随着价格的变化，持有的token的美元价值以及两种token的持有比例会大幅度变动。

但为了简化，在调整头寸的时候我们假设每次的价值都为初始头寸的价值，并且收集到的手续费不在调整的时候成为流动性。

**todo** 头寸token的美元价值动态调整

已知upper tick，lower tick，position的总价值，需要计算liquidity。

计算liquidity的公式：

```python
l1 = (token0_amount * (price_low ** 0.5) * (price_upper ** 0.5)) / ((price_upper ** 0.5) - (price_low ** 0.5))
l2 = token1_amount / ((price_upper ** 0.5) - (price_low ** 0.5))
l = min(l1, l2)
```
其中，token0_amount为5000，token1_amount为5000/price_token1

In [ ]:
# 策略是在当前tick下提供流动性，选取3为长度好了，tick范围是[currentTick-3, currentTick+3]
# 初始头寸的美元价值为10000usdc
initial_position = 10000

liquidity_data["position_token0_amount"] = 5000
liquidity_data["position_token1_amount"] = 5000  / liquidity_data["real_price_token1"]
liquidity_data["real_price_token1"] = 1 / (utils.price2RealPrice(liquidity_data["real_price"], 6, 18))
liquidity_data["upperTickIndex"] = liquidity_data["currentTickIndex"] + 3
liquidity_data["lowerTickIndex"] = liquidity_data["currentTickIndex"] - 3

In [125]:
liquidity_data["upperPrice"] = liquidity_data["upperTickIndex"].apply(lambda x: utils.tickIndex2Price(x))

In [127]:
liquidity_data["lowerPrice"] = liquidity_data["lowerTickIndex"].apply(lambda x: utils.tickIndex2Price(x))

In [140]:
liquidity_data.head()

,periodStartUnix,liquidity,sqrtPrice,token0Price,token1Price,tick,feeGrowthGlobal0X128,feeGrowthGlobal1X128,tvlUSD,volumeToken0,...,currentTickIndex,nearestTick,position_token0_amount,real_price_token1,position_token1_amount,upperTickIndex,lowerTickIndex,upperPrice,lowerPrice,my_liquidity
0,1679158800,31691854789759819627,1.871667e+33,1791.851542392230339967547914533666,0.0005580819483878331967604985081167192,201410,2314975469199928457298700227744982,1111162292302232016625192018264273618935405,362517939.1226720089381000671278629,0,...,201410.233545,201410.0,5000,1791.851542,2.790410,201413.233545,201407.233545,5.582494e+08,5.579146e+08,5002.790410
1,1679162400,32915185914729732691,1.867893e+33,1799.098710137099104209582500255936,0.0005558338707962253356429190522474716,201369,2315038215475779131181192686558395,1111189202831556094328153678234461295869361,363811618.6075028876192885997227013,0,...,201369.867963,201370.0,5000,1799.098710,2.779169,201372.867963,201366.867963,5.560006e+08,5.556672e+08,5002.779169
2,1679166000,34369179026466248085,1.869483e+33,1796.040997899866322473028892703993,0.0005567801632419932350116030494839081,201386,2315126854003039415933596971303595,1111241826769754789231620166343530092470985,363967492.8611761592527996744286012,0,...,201386.879073,201390.0,5000,1796.040998,2.783901,201389.879073,201383.879073,5.569472e+08,5.566132e+08,5002.783901
3,1679169600,34183163223601833767,1.868781e+33,1797.389578204559120182790388692096,0.000556362411424970994583196746389777,201379,2315168855482994101747218874079807,1111263525863761589158339866419647898430929,363205815.9317762712257394030005428,0,...,201379.372887,201380.0,5000,1797.389578,2.781812,201382.372887,201376.372887,5.565293e+08,5.561955e+08,5002.781812
4,1679173200,34246952167731560476,1.875753e+33,1784.054320332921814271944303424757,0.0005605210495011106585938907612211571,201453,2315193497548537858877306471738285,1111292420021385719852093400706179840724576,360909125.4200933323216227809336806,0,...,201453.845567,201450.0,5000,1784.054320,2.802605,201456.845567,201450.845567,5.606892e+08,5.603529e+08,5002.802605


In [190]:
liquidity_data["position_liquidity"] = 0
for i in range(0,liquidity_data.shape[0]):
    print("this is line : ", i)
    liquidity_data["position_liquidity"][i] = utils.calculateLiquidity(token0_amount, token1_amount,liquidity_data["lowerPrice"][i], liquidity_data["upperPrice"][i])

this is line :  0


KeyError: 'lowerPrice'

In [172]:
liquidity_data.head(1)

,periodStartUnix,liquidity,sqrtPrice,token0Price,token1Price,tick,feeGrowthGlobal0X128,feeGrowthGlobal1X128,tvlUSD,volumeToken0,...,position_token0_amount,real_price_token1,position_token1_amount,upperTickIndex,lowerTickIndex,upperPrice,lowerPrice,my_liquidity,my_liq,position_liquidity
0,1679158800,31691854789759819627,1.871667e+33,1791.851542392230339967547914533666,0.0005580819483878331967604985081167192,201410,2314975469199928457298700227744982,1111162292302232016625192018264273618935405,362517939.1226720089381000671278629,0,...,5000,1791.851542,2.79041,201413.233545,201407.233545,5.582494e+08,5.579146e+08,0.393749,1.116164e+09,0.393749


In [178]:
liquidity_data.dtypes

periodStartUnix             int64
liquidity                  object
sqrtPrice                 float64
token0Price                object
token1Price                object
tick                       object
feeGrowthGlobal0X128       object
feeGrowthGlobal1X128       object
tvlUSD                     object
volumeToken0               object
volumeToken1               object
volumeUSD                  object
feesUSD                    object
txCount                    object
open                       object
high                       object
low                        object
close                      object
real_price                float64
currentTickIndex          float64
nearestTick               float64
position_token0_amount      int64
real_price_token1         float64
position_token1_amount    float64
upperTickIndex            float64
lowerTickIndex            float64
upperPrice                float64
lowerPrice                float64
my_liquidity              float64
my_liq        

### 计算fee

现在有了liquidity的时间序列数据，可以计算头寸的手续费了。

In [186]:
# 获取所有swaps
swaps = query.query_swaps(begin=int(begin.timestamp()), end=int(end.timestamp()), pool_id=pool_id)

Error: Failed to get entities from store: canceling statement due to conflict with recovery, query = /* controller='filter',application='sgd217942',route='91044f11ec7c3947-475fe2ae7017c262',action='17071184' */
select 'Swap' as entity, to_jsonb(c.*) as data from (select  c.*
  from "sgd217942"."swap" c
 where c.block_range @> $1 and (c."id" > $2 and c."pool" = $3 and c."sqrt_price_x96" >= $4::numeric and c."sqrt_price_x96" <= $5::numeric and c."timestamp" > $6::numeric and c."timestamp" < $7::numeric)

 order by "id", block_range
 limit 1000) c -- binds: [17071184, "0x29a06069823ab60990bdfe428bd6a6bbd5fefbe9389874890e2724b1eed3cda6#4680692", "0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640", "0", "6277101735386680763835789423207666416102355444464034512896", "1679155200", "1681654435"]


In [187]:
swaps

,id,timestamp,sender,recipient,origin,amount0,amount1,amountUSD,sqrtPriceX96,tick,logIndex
0,0x00005be3feffa40600371196f6ebe99484cef2e030cb...,1679680919,0xef1c6e67703c7bd7107eed8303fbe6ec2554bf6b,0x697d1eeb266aec5251901e20e6a09dbdb68008c1,0x697d1eeb266aec5251901e20e6a09dbdb68008c1,-19922.526611,11.442382101497925129,19903.00432076901806488783284947735,1898284365247915042890667106569354,201692,295
1,0x00005e650a5a29a7340961ee1070bcb4d699a5c6b47c...,1680218951,0xef1c6e67703c7bd7107eed8303fbe6ec2554bf6b,0xef1c6e67703c7bd7107eed8303fbe6ec2554bf6b,0xc0c441bd9889f30fd219478a985e84fac8c2a706,1104.872406,-0.617085078176457911,1103.966933243716954948609702966833,1872856442624146468221690277382731,201422,369
2,0x00007ee86c8dedf910e254eb499746b9094d0d91ad3d...,1681392455,0x1111111254eeb25477b68fb85ed929f73a960582,0xbd4dbe0cb9136ffb4955ede88ebd5e92222ad09a,0x9d263af3950c80e3b49abd6def91f803762d65ab,-3951.498533,1.973163266992487917,3951.598149058666973024737265603912,1769998147893954485918035417674482,200293,75
3,0x00010099f534a61a99fa906e57cabd736e9f7e43ee0f...,1679774651,0xef1c6e67703c7bd7107eed8303fbe6ec2554bf6b,0xd22125cb225f39fe6aa0c16afe6b75872a3529d1,0xd22125cb225f39fe6aa0c16afe6b75872a3529d1,-12033.180287,7,12040.45320764383144992175201712345,1910435929075656551647679638334639,201820,272
4,0x00012828605998001bc178c99f7d95c467c7f481db6e...,1681089035,0xef1c6e67703c7bd7107eed8303fbe6ec2554bf6b,0xef1c6e67703c7bd7107eed8303fbe6ec2554bf6b,0x6992435f9a7bf34d5dedcad401e6ce8067cbb796,3299.547442,-1.774376095399703369,3302.745106341691833212028810929626,1837736537068722335967806380211673,201044,354
...,...,...,...,...,...,...,...,...,...,...,...
27995,0x299f094d81f8fc8955ba765d831f5e8f644d2ce3be4b...,1681403459,0xa83fbe4350c7443e6d0a21845aa786db96f0d6cf,0x1ac1a8feaaea1900c4166deeed0c11cc10669d36,0xf68660ad578ac43cbfbd39a9ba5b6a9d9fb009eb,-25889.429214,12.950000000000001,25874.12130714724085365066032757147,1771541426122364077843857814502206,200310,54
27996,0x299fdbdaa826837e02283f5e1121e4a1586b7152bb30...,1679552591,0x1111111254eeb25477b68fb85ed929f73a960582,0x1111111254eeb25477b68fb85ed929f73a960582,0xa10edf2eb42b5813bc801c0c9a374fe89d6b5a53,-1457.22203,0.829955019474836503,1455.794121457831405391317820361566,1890321956674315863521826452346330,201608,503
27997,0x299fea48be05a2a5965144dc6571aa841768de4dbe1a...,1680077663,0x9507c04b10486547584c37bcbd931b2a4fee9a41,0x9507c04b10486547584c37bcbd931b2a4fee9a41,0x75995f240e6eee8e80ce5898cdb3960da06aaa54,153735.041094,-84.767281439907083239,153503.4519551560925503083543032987,1860744933461097145715617130152772,201293,2
27998,0x29a0453abee3889377fcceede9312caa56b609cb50be...,1679315027,0xd7f3fbe8c72a961a5515203eada59750437fa762,0x1c09a10047fcc944efde9226e259eddfde2c1cf0,0x1c09a10047fcc944efde9226e259eddfde2c1cf0,232109.273443,-130,231948.1108427435531212720822827934,1875293919856857272249713373894412,201448,2


In [188]:
# 为了简化 假设主动调整头寸的操作足够及时，每一次swap都在我们的头寸之内，所以swaps不需要筛选
liquidity_data = liquidity_data[["periodStartUnix", "ratio"]]
liquidity_data = liquidity_data.rename(columns={"periodStartUnix": "timestamp", "ratio": "ratio"})
# 先排序
swaps["timestamp"] = swaps["timestamp"].astype(int)
swaps = swaps.sort_values(by="timestamp")
liquidity_data = liquidity_data.sort_values(by="timestamp")
# 找liquidity data只时间最接近的来合并
merged_data = pd.merge_asof(
    swaps,
    liquidity_data[["timestamp", "ratio"]],
    on="timestamp",
    direction="nearest"
)
merged_data.head(3)

,id,timestamp,sender,recipient,origin,amount0,amount1,amountUSD,sqrtPriceX96,tick,logIndex,ratio
0,0x0861d00acafcd74202e82741c031db7417fbca4c45fe...,1679155223,0xef1c6e67703c7bd7107eed8303fbe6ec2554bf6b,0xe63df014b8481ecc14582ff09e88809b0ad4e5a9,0xe63df014b8481ecc14582ff09e88809b0ad4e5a9,-17615.76052,9.835103915662891668,17613.78905750513568184175674883613,1871596376831572468181234428468238,201409,17,1.242430e-20
1,0x060ca03d50d79c0196c3b46dcbebb3dfef605b4f815e...,1679155343,0xe592427a0aece92de3edee1f18e0157c05861564,0x9008d19f58aabd9ed0d60971565aa8510560ab41,0xbff9a1b539516f9e20c7b621163e676949959a66,5994.484682,-3.349237752536633017,5995.99888113471748052423946471036,1873201169334335818044136888319180,201426,168,1.242430e-20
2,0x00a4333a3aad92b9579333e786936077d987a2022a6f...,1679155355,0xef1c6e67703c7bd7107eed8303fbe6ec2554bf6b,0xef1c6e67703c7bd7107eed8303fbe6ec2554bf6b,0xa62a7a20c38fd87d0ba3f06bfc84a3bce7596120,619.1124,-0.345987404437934662,619.337728969297592786277072330285,1873412840676047030438895530061802,201428,299,1.242430e-20


In [189]:
# 计算手续费
liquidity_data["ratio"] = liquidity_data["ratio"].astype(float)
merged_data['amount0'] = merged_data['amount0'].astype(float).apply(lambda x: max(0, x))
merged_data['amount1'] = merged_data['amount1'].astype(float).apply(lambda x: max(0, x))

merged_data["fee0"] = merged_data["amount0"] * merged_data['ratio'] * 0.0005
fee0_sum = merged_data["fee0"].sum()

merged_data["fee1"] = merged_data["amount1"] * merged_data['ratio'] * 0.0005
fee1_sum = merged_data["fee1"].sum()

print(fee0_sum)
print(fee1_sum)

print(fee0_sum)
print(fee1_sum)




6.3997272166463764e-15
3.4083994209935483e-18
6.3997272166463764e-15
3.4083994209935483e-18
